In [1]:
import pandas as pd
import json
from collections import defaultdict
from diverse_perturb import PerturbationCluster, PerturbationGenerator

full_df = pd.read_pickle("/Users/yeyiran/PycharmProjects/Norm_Pert/bert_important_word_list.pkl") 

with open('/Users/yeyiran/PycharmProjects/Norm_Pert/hard_pert_test.json') as hard_pert_f:
    old_hard_pert = json.load(hard_pert_f)

with open('/Users/yeyiran/PycharmProjects/Norm_Pert/easy_pert_test.json') as easy_pert_f:
    easy_pert = json.load(easy_pert_f)


with open('/Users/yeyiran/PycharmProjects/Norm_Pert/rule_pert_dict.json') as rule_pert_f:
    rule_pert = json.load(rule_pert_f)

hard_pert = defaultdict(list)
for k,v in old_hard_pert.items():
    for value in v:
        if value.lower() != k:
            hard_pert[k].append(value)


pert_total = defaultdict(set)
for k,v in old_hard_pert.items():
    for val in v:
        pert_total[k].add(val)

for k,v in easy_pert.items():
    for val in v:
        pert_total[k].add(val)

for k,v in rule_pert.items():
    for val in v[0]:
        pert_total[k].add(val)


Cluster = PerturbationCluster(pert_dict = pert_total, discard_func=['placeholder'])
cluster_dict = Cluster.classify()
attribute_list = Cluster.get_attribute_list()
Generator = PerturbationGenerator(cluster_dict,attribute_list, initial_cnt=[1,1,1,1,1,1])
Generator.shuffle()

In [3]:
MAX_NUM_OF_PERT = 1
THRESHOLD = 0.5
# num_of_idx = 0
cols = ['created_date', 'publication_id',
       'parent_id', 'article_id', 'rating', 'funny', 'wow', 'sad', 'likes',
       'disagree', 'toxicity', 'severe_toxicity', 'obscene', 'sexual_explicit',
       'identity_attack', 'insult', 'threat', 'male', 'female', 'transgender',
       'other_gender', 'heterosexual', 'homosexual_gay_or_lesbian', 'bisexual',
       'other_sexual_orientation', 'christian', 'jewish', 'muslim', 'hindu',
       'buddhist', 'atheist', 'other_religion', 'black', 'white', 'asian',
       'latino', 'other_race_or_ethnicity', 'physical_disability',
       'intellectual_or_learning_disability', 'psychiatric_or_mental_illness',
       'other_disability', 'identity_annotator_count',
       'toxicity_annotator_count']
empty_dict = {col:[] for col in cols}
empty_dict['clean'] = []
empty_dict['perturbed'] = []
result = pd.DataFrame.from_dict(data=empty_dict)
for idx,row in full_df.iterrows():
    # if not num_of_idx % 1000:
    #     print(num_of_idx, result.shape)
    # num_of_idx += 1
    clean = row['clean']
    word_list = clean.split()
    diff_weights = row['diff_weight_list']
    weight, location = diff_weights[0]
    if weight > THRESHOLD:
        pertbed_list = set()
        for i in range(MAX_NUM_OF_PERT):
            pert = Generator.pert_one(word_list[location])
            if pert:
                pertbed_list.add(' '.join(word_list[:location] + [pert] + word_list[location+1:]))

        for p in pertbed_list:
            cur_dict = {col:row[col] for col in cols}
            cur_dict['clean'] = [clean]
            cur_dict['perturbed'] = [p]
            result = pd.concat([result, pd.DataFrame(data=cur_dict)])



result

,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,...,latino,other_race_or_ethnicity,physical_disability,intellectual_or_learning_disability,psychiatric_or_mental_illness,other_disability,identity_annotator_count,toxicity_annotator_count,clean,perturbed
0,2017-09-08 06:10:57.978558+00,54.0,5918568.0,375121.0,approved,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,65.0,hit em low,hiT em low
0,2017-03-08 16:41:59.402377+00,13.0,NaN,317616.0,approved,0.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,52.0,there are some incredibly stupid people on thi...,there are some incredibly stpid people on this...
0,2017-10-18 22:58:40.549659+00,54.0,NaN,390678.0,approved,0.0,0.0,0.0,4.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,10.0,stupidity is increasing exponentially on plane...,stupidityI is increasing exponentially on plan...
0,2017-09-09 13:32:57.653593+00,102.0,5924299.0,375710.0,rejected,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,64.0,what a moron,what a morooon
0,2016-12-22 19:54:31.719790+00,54.0,NaN,157976.0,approved,0.0,0.0,0.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,64.0,the author of this little piece of trashy fict...,the author of this little piece of trashI fict...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2017-08-21 06:40:09.220587+00,21.0,5809395.0,368704.0,rejected,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,10.0,for justice we need the death penalty in a cas...,for justice we need the deaath penalty in a ca...
0,2016-02-12 21:30:11.686130+00,6.0,NaN,37721.0,approved,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,10.0,this is pretty much exactly how i feel damn,this is pretty much exactly how i feel daaammnn
0,2017-08-19 03:05:38.751272+00,21.0,NaN,368077.0,approved,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,10.0,a controversial rat,a controversial Ratt
0,2017-09-11 16:44:36.203924+00,105.0,NaN,376464.0,approved,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,10.0,more leftist insanity,more leftist 1nsanity


In [15]:
print(Generator.get_cnt())

lowercase_uppercase: 475
interesting_lowercase_uppercase: 230
emoticons: 0
repeat_char: 717
abbr: 373
special_character: 325


In [8]:
import time
import sys
sys.path.append('../')
from googleapiclient import discovery
from config import PERSPECTIVE_API_KEY
from serpapi import GoogleSearch
import pandas as pd 
import json

print(PERSPECTIVE_API_KEY)

client = discovery.build(
    "commentanalyzer",
    "v1alpha1",
    developerKey=PERSPECTIVE_API_KEY,
    discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
    static_discovery=False,
)

def perspective_acc(data_list,t=0.5):
    denominator = 0
    total = 0
    # for sentence in data_list:
    for i in range(len(data_list)):
        sentence = data_list[i]
        time.sleep(2)
        total += 1
        # print(total)

        analyze_request = {
            'comment': { 'text': sentence },
            'requestedAttributes': {'TOXICITY': {}},
            'languages': ["en"],
            'doNotStore':True
        }
        
        response = client.comments().analyze(body=analyze_request).execute()
        score = 0 if response["attributeScores"]["TOXICITY"]["spanScores"][0]["score"]["value"] < t else 1
        if score:
            denominator += 1
    return f"acc: {round(denominator/total,2)}"

# print(perspective_acc(list(result['perturbed']))) # 0.62
print(perspective_acc(list(result['clean']))) # 0.78


acc: 0.78


In [14]:
import numpy as np
import os, os.path
batch_size = 10
nparr = np.array_split(result, batch_size)
for i in range(batch_size):
    # cur = pd.concat([nparr[i], control], ignore_index=True)
    cur = nparr[i]
    path = f"batch_diverse/batch_{i}/"
    os.makedirs(os.path.dirname(path), exist_ok=True)
    cur.to_csv(path + "/raw.csv", index=False)
    print(cur.shape)

(212, 45)
(212, 45)
(212, 45)
(212, 45)
(212, 45)
(212, 45)
(212, 45)
(212, 45)
(212, 45)
(212, 45)


In [10]:
result.to_csv("diverse_pert_result.csv")